<a href="https://colab.research.google.com/github/abhinava-saha/abhinava-saha.github.io/blob/master/AnalyticsVidya_Hackathon/TensorFlow_AnalyticsVidya_Job_a_Thon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve,\
                            classification_report, confusion_matrix, plot_confusion_matrix

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # ‘all’|’last’|’last_expr’|’none’

from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/My Drive/DLCP"
root_path = '/content/drive/My Drive/DLCP'
 
#base_dir = root_dir + 'my-images/'

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/My Drive/DLCP/Hackathon/Train.csv')
test = pd.read_csv('/content/drive/My Drive/DLCP/Hackathon/Test.csv')

In [ ]:
from google.colab import files 
uploaded = files.upload() # upload file here from local

Saving submission_file_backup.csv to submission_file_backup (1).csv
Saving test_normalized.csv to test_normalized (1).csv
Saving train_normalized.csv to train_normalized.csv


In [ ]:
import io
df_train = pd.read_csv(io.BytesIO(uploaded['train_normalized.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['test_normalized.csv']))
test_submission = pd.read_csv(io.BytesIO(uploaded['submission_file_backup.csv']))

In [ ]:
df_train = pd.read_csv('train_normalized.csv')
df_test = pd.read_csv('test_normalized.csv')
test_submission = pd.read_csv('submission_file_backup.csv')

In [ ]:
df_train.head()
df_test.head()
test_submission.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,0,0.806452,18,2,2,0.281250,0.0,6.919500,0,0
1,0,0.112903,27,1,0,0.195312,0.0,6.796826,0,0
2,0,0.532258,18,0,2,0.148438,0.0,6.988967,1,0
3,1,0.177419,20,1,0,0.093750,0.0,6.750197,0,0
4,0,0.112903,32,1,0,0.203125,0.0,6.885826,0,0


,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,1,0.096774,4,2,0,0.140625,1.0,6.848787,0
1,1,0.322581,18,2,1,0.328125,0.0,6.894625,0
2,1,0.129032,20,1,0,0.054688,0.0,6.569445,0
3,1,0.096774,22,2,0,0.203125,0.0,6.881422,0
4,0,0.096774,20,2,0,0.093750,0.0,6.822919,0


,ID,Is_Lead
0,VBENBARO,0.035603
1,CCMEWNKY,0.264289
2,VK3KGA9M,0.067893
3,TT8RPZVC,0.034360
4,SHQZEYTZ,0.032349


In [ ]:
X = df_train.drop(['Is_Lead'],axis=1)
y = df_train[['Is_Lead']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=51,stratify=df_train['Is_Lead'])


print(f'\033[1mSpliting the Company dataset in Ratio 70:30. \nShape of ---> \033[0m\n\033[34mX_Train:{X_train.shape}, \ny_train:{y_train.shape},\
      \nX_Test:{X_test.shape}, \ny_test:{y_test.shape}\033[0m')


Spliting the Company dataset in Ratio 70:30. 
Shape of ---> 
X_Train:(172007, 9), 
y_train:(172007, 1),      
X_Test:(73718, 9), 
y_test:(73718, 1)


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        DESIRED_ACCURACY = 0.90
        if(logs.get('auc') is not None and logs.get('auc') > DESIRED_ACCURACY):
            print("Reached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=15,activation='relu', input_shape=(X_train.shape[-1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
                            ])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 15)                150       
_________________________________________________________________
dropout_1 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 16        
Total params: 166
Trainable params: 166
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=1e-3),
              metrics=[METRICS])

In [55]:
history = model.fit(
        x=X_train, y=y_train,
#      steps_per_epoch=8,  
        epochs=200,
        callbacks=[callbacks],
        verbose=1,
#        class_weight={0:0.3, 1:0.7},
        validation_data = (X_test, y_test),
        validation_steps=3)

Epoch 1/200
5376/5376 [==============================] - 20s 3ms/step - loss: 0.5993 - tp: 1432.0000 - fp: 4558.0000 - tn: 126648.0000 - fn: 39369.0000 - accuracy: 0.7446 - precision: 0.2391 - recall: 0.0351 - auc: 0.6245 - prc: 0.2949 - val_loss: 0.4937 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 56231.0000 - val_fn: 17487.0000 - val_accuracy: 0.7628 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7087 - val_prc: 0.3748
Epoch 2/200
5376/5376 [==============================] - 16s 3ms/step - loss: 0.5025 - tp: 292.0000 - fp: 299.0000 - tn: 130907.0000 - fn: 40509.0000 - accuracy: 0.7628 - precision: 0.4941 - recall: 0.0072 - auc: 0.6887 - prc: 0.3635 - val_loss: 0.4899 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 56231.0000 - val_fn: 17487.0000 - val_accuracy: 0.7628 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7193 - val_prc: 0.3918
Epoch 3/200
5376/5376 [==============================] - 15s 3ms/step - loss: 0.4992 - tp: 708.0000 

In [ ]:
classes = model.predict(X_test)